In [3]:
#Interactive plot for the directivity shape estimation
#Jithin Thilakan, August 1, 2022

import numpy as np
import math
import cmath
import plotly.graph_objects as go

from ipywidgets import interact, interactive, fixed, interact_manual

c = 340                               # speed of sound in m/s
Dp = (47.5+48.5+46.5+48.2)/4000       # average diameter of the pipe in m at the passive end
Rp = Dp/2                             # radius of the pipe in m 
Ap = (math.pi)*(Rp)**2                # pipe area at passive end in m^2
h = 11.2e-3                           # heigth of the mouth in m
w = 35.8e-3                           # width of the mouth in m
Am = h*w                              # area of the mouth in c^2
dlp=0.34*(math.sqrt(Ap))              # passive end correction in m
dlm=0.27*Ap/(math.sqrt(Am))           # mouth correction in m
Q = 1e-3                              # amplitude of oscillator
df = 1                                # frequency increment in Hz
rho0 = 1.2041                         # density of air in kg/m^3

def Levelvaluator(freq,dynrange,pipe_length,res):
    Lp = pipe_length/100                  # lenth of the pipe in m
    L = Lp-h+dlm+dlp                      # length of e.g. flute from foot to passive end in z-direction in m  
    #res                                  # resolution of grid
    dphi = int(360/res/2+1)               # increment phi 37@res°
    dtheta = int(360/res+1)               # increment theta 73@res°
    
    def pressure_estimator(amp,f,D,Xs,Ys,Zs,Xo,Yo,Zo,a,dphi,dtheta): 
        #CAUTION; The coordinates are (Xs,Ys,Zs) now for the source; not (Z,Y,X)
        #give amplitude-amp, frequency-f, source location-(Xs,Yss,Z), observation location-(xo,yo,zo), and radius of tonehole-a

        delta_Phi = D*f*2*(math.pi)/c
        R_square = (Xo-Xs)**2+(Yo-Ys)**2+(Zo-Zs)**2
        delta_R = math.sqrt(R_square)

        #pressure estimation equation
        term1 = (1j*2*(math.pi)*f*rho0*amp)/(4*(math.pi)*delta_R)

        power_term = 1j*((-2*(math.pi)*f*delta_R/c)-delta_Phi)

        term2 = cmath.exp(power_term)
       # term2 = 2.71828**(power_term)

        P_early = term1*term2
        return P_early


    #pressure_val = pressure_estimator(Q,400,L,0,0,L/2,1,1,1,0,dphi,dtheta)  #to double check the function output

    r=1 #observation radius in m

    phi = np.linspace(-(math.pi)/2, (math.pi)/2, dphi)     #elevation; between -pi/2 to pi/2 with res degree resolution
    theta = np.linspace(0, 2*(math.pi), dtheta)    #azimuth; between 0 to 2pi with res degree spacing
    phi_mat, theta_mat = np.meshgrid(phi, theta, indexing='ij')
       
    #Function for converting spherical to cartesian coordinates

    def sph2cart(az, el, r):
        z = r * np.sin(el)
        rcos_el = r * np.cos(el)
        x = rcos_el * np.cos(az)
        y = rcos_el * np.sin(az)
        return [x, y, z]

    #estimating the cartesian location of all the grid points
    cart_coordinate=[]

    for i in range(0, dphi):
        for j in range(0, dtheta):

          cart_val = sph2cart(theta_mat[i,j],phi_mat[i,j],r)
          cart_coordinate.append(cart_val)

    grid_points = np.array(cart_coordinate)
    k= len(grid_points)

    pressure_values =[]

    for i in range(0, k):
        xo = grid_points[i,0]
        yo = grid_points[i,1]
        zo = grid_points[i,2]

        pressure1 = pressure_estimator(Q,freq,0,0,0,-L/2,xo,yo,zo,0,dphi,dtheta)  #source 1
        pressure2 = pressure_estimator(Q,freq,L,0,0,L/2,xo,yo,zo,0,dphi,dtheta)   #source 2
        pressuresum =pressure1+pressure2
        pressure_values.append(pressuresum)


    p_early = np.array(pressure_values)

    p_ratio = abs(p_early)/(2*(math.pow(10,-5)))


    SPL_mat=np.zeros(k)
    for i in range(0, k):
        SPL=20*(math.log(p_ratio[i],10))
        SPL_mat[i]=SPL

    maxL = max(SPL_mat)

    L_mat = np.zeros(k)
    for i in range(0, k):
        Level = (SPL_mat[i])-maxL+dynrange
        L_mat[i] = Level

    L_mat[L_mat < 0] = 0

    Level_mat=np.reshape(L_mat, (dphi, dtheta))

    #estimating the Level values from spherical to cartesian
    Lvalue_cartesianvalues=[]

    for i in range(0, dphi):
        for j in range(0, dtheta):

          L_xyz = sph2cart(theta_mat[i,j],phi_mat[i,j],Level_mat[i,j])
          Lvalue_cartesianvalues.append(L_xyz)
        
        Lvalue_cart_mat= np.array(Lvalue_cartesianvalues)

    grid_points = np.array(cart_coordinate)
    
    X_plot=Lvalue_cart_mat[:,0]
    Y_plot=Lvalue_cart_mat[:,1]
    Z_plot=Lvalue_cart_mat[:,2]

    X_plot=np.reshape(X_plot, (dphi, dtheta))
    Y_plot=np.reshape(Y_plot, (dphi, dtheta))
    Z_plot=np.reshape(Z_plot, (dphi, dtheta))
    
 
    #PLOTTING USING PLOTLY
    fig = go.Figure(data=[go.Surface(z=Z_plot, x=X_plot, y=Y_plot, surfacecolor=Level_mat)])
    fig.update_layout(title='Directivity pattern', autosize=False, 
                  width=900, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))
    fig.show()
       
interact(Levelvaluator, freq=(10,4000), dynrange=(0,80), pipe_length=(0,100), res=(1,10))


interactive(children=(IntSlider(value=2005, description='freq', max=4000, min=10), IntSlider(value=40, descrip…

<function __main__.Levelvaluator(freq, dynrange, pipe_length, res)>